# Score SNPs

Use trained models to score SNPs.

In [17]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pyfaidx
from collections import Counter, defaultdict, OrderedDict
from modisco.visualization import viz_sequence
import logomaker
from copy import deepcopy
from tqdm import tqdm

import sys
sys.path.append("/scratch/dyl3pc/BPNet/retina-models/src")
from utils.loss import multinomial_nll
from utils import one_hot
from utils.data_utils import load_test_data, get_seq
from utils.shap_utils import shuffle_several_times, combine_mult_and_diffref

import shap
import glob
tf.compat.v1.disable_eager_execution()

from metrics import softmax 
import scipy.stats
from mhc import multiple_testing_correction

import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
def get_profile(output):
    prof, cts = output
    return softmax(prof)*(np.exp(cts)-1)

## Load Models

In [4]:
hg38 = pyfaidx.Fasta("/scratch/dyl3pc/Clint/resources/hg38_UCSC.fa")

In [5]:
models = defaultdict(dict)
ctypes = [os.path.basename(dir_) for dir_ in glob.glob("/scratch/dyl3pc/BPNet/output/pos_regions/*")]
FOLDS = [1,2,3,4,5]
with tf.keras.utils.CustomObjectScope({'multinomial_nll':multinomial_nll, 'tf':tf}):
    for ctype in ctypes:
        for fold in FOLDS:
            models[ctype][fold] = tf.keras.models.load_model(f"/scratch/dyl3pc/BPNet/output/model/{ctype}/fold{fold}.h5")

2022-05-01 10:58:26.190797: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-05-01 10:58:26.246501: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:03:00.0 name: Tesla P100-PCIE-12GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 11.91GiB deviceMemoryBandwidth: 511.41GiB/s
2022-05-01 10:58:26.246571: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2022-05-01 10:58:28.017207: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2022-05-01 10:58:28.017375: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11
2022-05-01 10:58:29.090265: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuff

In [6]:
model_count_explainers = defaultdict(dict)

# explainers
for x in ctypes:
    for i in FOLDS:
        model_count_explainers[x][i] = shap.explainers.deep.TFDeepExplainer(
                (models[x][i].input,
                 tf.reduce_sum(models[x][i].outputs[1], axis=-1)),
                shuffle_several_times,
                combine_mult_and_diffref=combine_mult_and_diffref)

In [7]:
INP_LEN = models[ctypes[0]][1].input_shape[1]
OUT_LEN = models[ctypes[0]][1].output_shape[0][1]

INP_LEN, OUT_LEN

(2114, 1000)

In [8]:
snp_lists = dict()

for x in glob.glob("/scratch/dyl3pc/BPNet/external_resources/*"):
    set_name = os.path.basename(x).split('.')[0]
    
    snp_lists[set_name] = pd.read_csv(x,
                       sep='\t',
                      names=['chr', 'start', 'rs', 'ref', 'alt'])

In [9]:
BUFFER = 500 # to adjust for indels

ref_one_hots = defaultdict(list)
alt_one_hots = defaultdict(list)
ref_mismatch = 0

for x in snp_lists:
#     print(x)
    for _, y in snp_lists[x].iterrows():
        ref_one_hots[x].append(str(hg38[y['chr']][(y['start'] - INP_LEN//2):(y['start'] + INP_LEN//2 + BUFFER)]))
        
        # correct those that don't match ref as per dataframe
        if ref_one_hots[x][-1][INP_LEN//2 - 1:][:len(y['ref'])] != y['ref']:
            ref_mismatch += 1
            ref_one_hots[x][-1] = ref_one_hots[x][-1][:INP_LEN//2 - 1] + y['ref'] + ref_one_hots[x][-1][INP_LEN//2 - 1 + len(y['ref']):]
            assert(ref_one_hots[x][-1][INP_LEN//2 - 1:][:len(y['ref'])] == y['ref'])
#             print(ref_one_hots[x][-1][INP_LEN//2 - 1:][:len(y['ref'])], y['ref'], y['alt'])
        
        cur_alt = ref_one_hots[x][-1]
        cur_alt = cur_alt[:INP_LEN//2 - 1] + y['alt'] + cur_alt[INP_LEN//2 -1 + len(y['ref']):]
        alt_one_hots[x].append(cur_alt)
        assert(alt_one_hots[x][-1][INP_LEN//2 - 1:][:len(y['alt'])] == y['alt'])
        
        # trim to model input length
        ref_one_hots[x][-1] = ref_one_hots[x][-1][:INP_LEN]
        alt_one_hots[x][-1] = alt_one_hots[x][-1][:INP_LEN]

    ref_one_hots[x] = one_hot.dna_to_one_hot(ref_one_hots[x])
    alt_one_hots[x] = one_hot.dna_to_one_hot(alt_one_hots[x])

In [10]:
# should be small compared to all SNPs
ref_mismatch

7869

## Score

In [11]:
ref_preds = defaultdict(lambda: defaultdict(dict))
alt_preds = defaultdict(lambda: defaultdict(dict))

for sl in snp_lists:
    print(sl)
    for m in tqdm(models):
        for i in FOLDS:
            ref_preds[sl][m][i] = models[m][i].predict(ref_one_hots[sl], verbose=True, batch_size=256)
            alt_preds[sl][m][i] = models[m][i].predict(alt_one_hots[sl], verbose=True, batch_size=256)
    print('------')

  0%|          | 0/9 [00:00<?, ?it/s]

CAD


/nv/vol192/civeleklab/dillon/conda/shap/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:2424: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
2022-05-01 11:03:41.232332: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2022-05-01 11:03:43.995348: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8201
2022-05-01 11:03:47.597019: W tensorflow/stream_executor/gpu/asm_compiler.cc:64] Running ptxas --version returned 32512
2022-05-01 11:03:47.725433: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 32512, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged o

------


## Aggregate across folds

In [13]:
def poisson_pval(ref, alt):
    # one sided (either test breaking peak or making peak), ref alt are log
    if ref <= 0:
        return 1
    
    if alt>ref:
        return 1-scipy.stats.poisson(np.exp(ref)-1).cdf(np.exp(alt)-1)
    else:
        return scipy.stats.poisson(np.exp(ref)-1).cdf(max(0,np.exp(alt)-1))

In [15]:
snp_score_dfs = dict()

for sl in snp_lists:
    cur_dict = OrderedDict()
    cur_dict['chr'] = snp_lists[sl]['chr']
    cur_dict['start'] = snp_lists[sl]['start']
    cur_dict['rs'] = snp_lists[sl]['rs']
    cur_dict['ref'] = snp_lists[sl]['ref']
    cur_dict['alt'] = snp_lists[sl]['alt']
    
    for m in models:
        print(m)
        all_folds = []
        refs = []
        alts = []
        
        for i in FOLDS:
            cur_lfc = (alt_preds[sl][m][i][1] - ref_preds[sl][m][i][1]).ravel() # simple counts lfc
            refs.append(ref_preds[sl][m][i][1].ravel())
            alts.append(alt_preds[sl][m][i][1].ravel())
        
            # JSD (fails for indels)
#             cur_lfc = [scipy.spatial.distance.jensenshannon(x,y) for x,y in zip(softmax(alt_preds[sl][m][i][0]), softmax(ref_preds[sl][m][i][0]))] # JSD 

            # counts LFC in central 100 bp
#             cur_lfc = np.log(1+get_profile(alt_preds[sl][m][i])[:, (OUT_LEN-100)//2:(OUT_LEN+100)//2].sum(-1)) - \
#                       np.log(1+get_profile(ref_preds[sl][m][i])[:, (OUT_LEN-100)//2:(OUT_LEN+100)//2].sum(-1))
        
            all_folds.append(list(cur_lfc))
        
        all_folds = np.array(all_folds).T
        mean_refs = np.array(refs).mean(0)
        mean_alts = np.array(alts).mean(0)
        
        cur_dict['{}_fold_avg_lfc'.format(m)] = all_folds.mean(-1)
        cur_dict['{}_avg_ref'.format(m)] = mean_refs
        cur_dict['{}_avg_alt'.format(m)] = mean_alts
        
        for i in FOLDS:
            cur_dict['{}_fold{}_lfc'.format(m,i)] = all_folds[:,i-1].copy()
        
        pvals = [poisson_pval(x,y) for x,y in zip(mean_refs, mean_alts)]
        cur_dict['{}_pval'.format(m)] = pvals
        
    snp_score_dfs[sl] = pd.DataFrame(cur_dict)

Pericyte
Plasma
unknown
Macrophage
T
SMC
Mast
Endothelial
Fibroblast


In [18]:
for sl in snp_lists:
    snp_score_dfs[sl]['fisher_fused_pval'] = snp_score_dfs[sl][['{}_pval'.format(m) for m in models]].\
                                                apply(lambda x: scipy.stats.combine_pvalues(x)[1], axis=1)
    snp_score_dfs[sl]['fdr'] = multiple_testing_correction(list(snp_score_dfs[sl]['fisher_fused_pval']))

In [19]:
for sl in snp_lists:
    snp_score_dfs[sl]['max_lfc'] = np.array(snp_score_dfs[sl][["{}_fold_avg_lfc".format(m) for m in models]].apply(lambda x: max(x, key=lambda y:abs(y)), 1))
    snp_score_dfs[sl]['max_log2fc'] = np.log2(np.exp(snp_score_dfs[sl]['max_lfc']))

In [20]:
LFC_CUTOFF = np.log(2**0.5)
FDR_CUTOFF = 0.01

for sl in snp_lists:
    snp_score_dfs[sl]['high_effect'] = np.int8((np.abs(snp_score_dfs[sl]['max_lfc']) > LFC_CUTOFF) & (snp_score_dfs[sl]['fdr'] < FDR_CUTOFF))

In [24]:
reordered_cols = list(snp_score_dfs['CAD'].columns)
reordered_cols = reordered_cols[:5] + reordered_cols[-5:] + reordered_cols[5:-5]

In [25]:
reordered_cols[:10]

['chr',
 'start',
 'rs',
 'ref',
 'alt',
 'fisher_fused_pval',
 'fdr',
 'max_lfc',
 'max_log2fc',
 'high_effect']

In [26]:
for sl in snp_lists:
    snp_score_dfs[sl] = snp_score_dfs[sl][reordered_cols]

In [42]:
snp_score_df = snp_score_dfs['CAD']

In [44]:
snp_score_df.to_csv("/project/cphg-millerlab/CAD_QTL/coronary_QTL/epigenome/scATAC/BPNet/all_folds.tsv", sep='\t', index=False)

In [47]:
cols = ["chr", "start", "rs", "ref", "alt", "fisher_fused_pval", "fdr", "max_lfc", "max_log2fc", "high_effect"]
cols += [f"{ctype}_fold_avg_lfc" for ctype in ctypes]
cols += [f"{ctype}_pval" for ctype in ctypes]
snp_score_avg_df = snp_score_df[cols]

In [49]:
snp_score_avg_df.to_csv("/project/cphg-millerlab/CAD_QTL/coronary_QTL/epigenome/scATAC/BPNet/avg_results.tsv", sep='\t', index=False)